# Capítulo 1: O Fim do Modelo "Volcano"

## Introdução

Durante décadas, o modelo **Volcano** (também conhecido como Iterator Model) foi o padrão de facto para execução de consultas em bancos de dados relacionais. Sistemas como PostgreSQL, MySQL e Oracle utilizam variações deste modelo.

No entanto, o DuckDB representa uma mudança de paradigma: ele utiliza o **Modelo Vetorizado**, que processa dados em blocos (vetores) ao invés de linha por linha.

### Objetivos deste capítulo:
1. Entender o modelo Volcano e suas limitações
2. Compreender o modelo vetorizado do DuckDB
3. Comparar performance entre os dois modelos
4. Visualizar o impacto na CPU e cache

## 1.1 O Modelo Volcano Tradicional

No modelo Volcano, cada operador implementa três métodos:
- `open()`: Inicializa o operador
- `next()`: Retorna a **próxima tupla** (uma linha por vez)
- `close()`: Libera recursos

### Problemas do Modelo Volcano:
1. **Overhead de chamadas de função**: Cada linha requer uma chamada virtual
2. **Cache misses**: Dados não ficam no cache L1/L2
3. **Branch misprediction**: CPU não consegue prever o fluxo
4. **Sem SIMD**: Não aproveita instruções vetoriais modernas

In [ ]:
# Instalação das dependências
!pip install duckdb pandas numpy matplotlib seaborn -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração visual
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 1.2 Simulando o Modelo Volcano em Python

Vamos criar uma simulação didática do modelo Volcano para entender o overhead de processar linha por linha.

In [ ]:
class VolcanoIterator:
    """
    Simulação do modelo Volcano - processa uma linha por vez
    """
    def __init__(self, data):
        self.data = data
        self.index = 0
    
    def open(self):
        self.index = 0
    
    def next(self):
        """Retorna próxima tupla - chamada de função por linha!"""
        if self.index >= len(self.data):
            return None
        row = self.data[self.index]
        self.index += 1
        return row
    
    def close(self):
        pass


class VolcanoSum(VolcanoIterator):
    """Operador de soma no modelo Volcano"""
    def __init__(self, child, col_a, col_b):
        self.child = child
        self.col_a = col_a
        self.col_b = col_b
    
    def open(self):
        self.child.open()
    
    def next(self):
        row = self.child.next()  # Chamada virtual!
        if row is None:
            return None
        # Processa UMA linha
        return row[self.col_a] + row[self.col_b]
    
    def close(self):
        self.child.close()

In [ ]:
# Teste do modelo Volcano
n_rows = 100_000
data = [(i, i*2) for i in range(n_rows)]  # Lista de tuplas (a, b)

def volcano_execution(data):
    """Executa consulta SELECT a + b usando modelo Volcano"""
    scan = VolcanoIterator(data)
    sum_op = VolcanoSum(scan, 0, 1)
    
    sum_op.open()
    results = []
    while True:
        result = sum_op.next()  # Uma chamada por linha!
        if result is None:
            break
        results.append(result)
    sum_op.close()
    return results

# Medindo tempo
start = time.perf_counter()
results_volcano = volcano_execution(data)
volcano_time = time.perf_counter() - start

print(f"Modelo Volcano: {volcano_time*1000:.2f} ms para {n_rows:,} linhas")
print(f"Chamadas de função next(): {n_rows:,}")

## 1.3 O Modelo Vetorizado

No modelo vetorizado, os dados são processados em **blocos** (vetores) de tamanho fixo (tipicamente 1024-2048 linhas).

### Vantagens:
1. **Menos chamadas de função**: Uma chamada processa milhares de linhas
2. **Cache friendly**: Dados sequenciais ficam no cache L1
3. **SIMD ready**: Operações podem usar instruções vetoriais
4. **Melhor branch prediction**: Loops internos são previsíveis

In [ ]:
class VectorizedOperator:
    """
    Simulação do modelo vetorizado - processa blocos de dados
    """
    VECTOR_SIZE = 1024  # Tamanho típico do DuckDB
    
    def __init__(self, data_a, data_b):
        self.data_a = np.array(data_a)
        self.data_b = np.array(data_b)
        self.offset = 0
    
    def next_chunk(self):
        """
        Retorna próximo chunk de dados (vetor de até 1024 elementos)
        Uma única chamada processa MUITAS linhas!
        """
        if self.offset >= len(self.data_a):
            return None
        
        end = min(self.offset + self.VECTOR_SIZE, len(self.data_a))
        chunk_a = self.data_a[self.offset:end]
        chunk_b = self.data_b[self.offset:end]
        self.offset = end
        
        # Operação vetorizada - SIMD friendly!
        # O compilador/NumPy traduz isso para instruções SIMD
        return chunk_a + chunk_b


def vectorized_execution(data_a, data_b):
    """Executa consulta SELECT a + b usando modelo vetorizado"""
    op = VectorizedOperator(data_a, data_b)
    
    results = []
    while True:
        chunk = op.next_chunk()  # Uma chamada por bloco de 1024!
        if chunk is None:
            break
        results.append(chunk)
    
    return np.concatenate(results)

In [ ]:
# Preparando dados para modelo vetorizado
data_a = [row[0] for row in data]
data_b = [row[1] for row in data]

# Medindo tempo do modelo vetorizado
start = time.perf_counter()
results_vectorized = vectorized_execution(data_a, data_b)
vectorized_time = time.perf_counter() - start

n_chunks = (n_rows + 1023) // 1024
print(f"Modelo Vetorizado: {vectorized_time*1000:.2f} ms para {n_rows:,} linhas")
print(f"Chamadas de função next_chunk(): {n_chunks}")
print(f"\nSpeedup: {volcano_time/vectorized_time:.1f}x mais rápido")

## 1.4 Comparação Visual: Volcano vs Vetorizado

In [ ]:
# Benchmark com diferentes tamanhos de dados
sizes = [1_000, 10_000, 50_000, 100_000, 500_000, 1_000_000]
volcano_times = []
vectorized_times = []

for size in sizes:
    # Gerar dados
    test_data = [(i, i*2) for i in range(size)]
    test_a = [row[0] for row in test_data]
    test_b = [row[1] for row in test_data]
    
    # Volcano
    start = time.perf_counter()
    _ = volcano_execution(test_data)
    volcano_times.append((time.perf_counter() - start) * 1000)
    
    # Vetorizado
    start = time.perf_counter()
    _ = vectorized_execution(test_a, test_b)
    vectorized_times.append((time.perf_counter() - start) * 1000)
    
    print(f"Size: {size:>10,} | Volcano: {volcano_times[-1]:>8.2f}ms | Vectorized: {vectorized_times[-1]:>8.2f}ms")

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de tempo absoluto
ax1 = axes[0]
x = range(len(sizes))
width = 0.35

bars1 = ax1.bar([i - width/2 for i in x], volcano_times, width, label='Volcano', color='#e74c3c')
bars2 = ax1.bar([i + width/2 for i in x], vectorized_times, width, label='Vetorizado', color='#27ae60')

ax1.set_xlabel('Número de Linhas')
ax1.set_ylabel('Tempo (ms)')
ax1.set_title('Tempo de Execução: Volcano vs Vetorizado')
ax1.set_xticks(x)
ax1.set_xticklabels([f'{s:,}' for s in sizes], rotation=45)
ax1.legend()
ax1.set_yscale('log')

# Gráfico de speedup
ax2 = axes[1]
speedups = [v/vec for v, vec in zip(volcano_times, vectorized_times)]
ax2.bar(x, speedups, color='#3498db')
ax2.set_xlabel('Número de Linhas')
ax2.set_ylabel('Speedup (x vezes mais rápido)')
ax2.set_title('Speedup do Modelo Vetorizado')
ax2.set_xticks(x)
ax2.set_xticklabels([f'{s:,}' for s in sizes], rotation=45)
ax2.axhline(y=1, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 1.5 DuckDB em Ação: Execução Vetorizada Real

Agora vamos ver o DuckDB executando consultas reais e comparar com processamento linha a linha em Python/Pandas.

In [ ]:
# Criar uma tabela grande no DuckDB
con = duckdb.connect(':memory:')

# Gerar dados de teste
n = 10_000_000  # 10 milhões de linhas

con.execute(f"""
    CREATE TABLE test AS
    SELECT 
        i AS id,
        random() * 1000 AS value_a,
        random() * 1000 AS value_b,
        CASE WHEN random() > 0.5 THEN 'A' ELSE 'B' END AS category
    FROM generate_series(1, {n}) AS t(i)
""")

print(f"Tabela criada com {n:,} linhas")
con.execute("SELECT * FROM test LIMIT 5").df()

In [ ]:
# Consulta: SELECT value_a + value_b FROM test
# Comparando DuckDB vs Pandas

# DuckDB (vetorizado)
start = time.perf_counter()
result_duck = con.execute("SELECT value_a + value_b AS soma FROM test").df()
duckdb_time = time.perf_counter() - start

print(f"DuckDB (Vetorizado): {duckdb_time*1000:.2f} ms")

In [ ]:
# Pandas (também vetorizado via NumPy, mas com overhead de DataFrame)
df = con.execute("SELECT * FROM test").df()

start = time.perf_counter()
result_pandas = df['value_a'] + df['value_b']
pandas_time = time.perf_counter() - start

print(f"Pandas: {pandas_time*1000:.2f} ms")
print(f"\nDuckDB é {pandas_time/duckdb_time:.1f}x mais rápido que Pandas para esta operação")

In [ ]:
# Simulação de processamento linha a linha (estilo Volcano em Python)
def row_by_row_sum(df):
    """Simula processamento linha por linha - MUITO lento!"""
    results = []
    for idx in range(len(df)):
        results.append(df.iloc[idx]['value_a'] + df.iloc[idx]['value_b'])
    return results

# Testar apenas com subset (seria muito lento com 10M linhas)
df_small = df.head(100_000)

start = time.perf_counter()
_ = row_by_row_sum(df_small)
row_time = time.perf_counter() - start

# Tempo vetorizado para mesma quantidade
start = time.perf_counter()
_ = df_small['value_a'] + df_small['value_b']
vec_time = time.perf_counter() - start

print(f"Linha a linha (100k rows): {row_time*1000:.2f} ms")
print(f"Vetorizado (100k rows): {vec_time*1000:.4f} ms")
print(f"\nVetorização é {row_time/vec_time:.0f}x mais rápida!")

## 1.6 Visualizando o Plano de Execução do DuckDB

O DuckDB permite visualizar como ele processa consultas internamente.

In [ ]:
# Ver o plano de execução
print("=== EXPLAIN ===\n")
plan = con.execute("EXPLAIN SELECT value_a + value_b FROM test WHERE category = 'A'").df()
print(plan.to_string())

In [ ]:
# Análise detalhada com EXPLAIN ANALYZE
print("=== EXPLAIN ANALYZE ===\n")
analyze = con.execute("EXPLAIN ANALYZE SELECT SUM(value_a + value_b) FROM test WHERE category = 'A'").df()
print(analyze.to_string())

## 1.7 Por Que Vetorização Funciona Tão Bem?

### Diagrama conceitual do cache da CPU:

```
+-------------------------------------------------------------+
|                         CPU                                  |
|  +-----------------------------------------------------+   |
|  |  Registradores (< 1ns)                               |   |
|  |  +-----------------------------------------------------+ |
|  |  |  Cache L1 (32KB, ~1ns) <-- Dados do Chunk       |   |
|  |  +-----------------------------------------------------+ |
|  |  +-----------------------------------------------------+ |
|  |  |  Cache L2 (256KB, ~3ns)                         |   |
|  |  +-----------------------------------------------------+ |
|  +-----------------------------------------------------+   |
|  +-----------------------------------------------------+   |
|  |  Cache L3 (8MB, ~10ns)                               |   |
|  +-----------------------------------------------------+   |
+-------------------------------------------------------------+
                          |
                          v (~100ns latência!)
+-------------------------------------------------------------+
|                     RAM Principal                            |
+-------------------------------------------------------------+
```

**O segredo**: Um chunk de 1024-2048 linhas com poucos campos cabe inteiramente no Cache L1!

In [ ]:
# Demonstração: tamanho de um chunk típico
chunk_size = 2048
bytes_per_double = 8  # float64
num_columns = 3

chunk_memory = chunk_size * bytes_per_double * num_columns
l1_cache_size = 32 * 1024  # 32 KB típico

print(f"Tamanho de um chunk (2048 linhas x 3 colunas float64):")
print(f"  {chunk_memory:,} bytes ({chunk_memory/1024:.1f} KB)")
print(f"\nCache L1 típico: {l1_cache_size:,} bytes ({l1_cache_size/1024:.0f} KB)")
print(f"\n O chunk cabe no L1? {'Sim!' if chunk_memory < l1_cache_size else 'Não'}")

## 1.8 Exercícios Práticos

### Exercício 1: Implemente um operador de filtro no modelo Volcano

In [ ]:
# TODO: Implemente a classe VolcanoFilter que filtra linhas onde coluna > threshold
class VolcanoFilter(VolcanoIterator):
    def __init__(self, child, column_idx, threshold):
        self.child = child
        self.column_idx = column_idx
        self.threshold = threshold
    
    def open(self):
        self.child.open()
    
    def next(self):
        # Sua implementação aqui
        while True:
            row = self.child.next()
            if row is None:
                return None
            if row[self.column_idx] > self.threshold:
                return row
    
    def close(self):
        self.child.close()

# Teste
test_data = [(i, i*2) for i in range(10)]
scan = VolcanoIterator(test_data)
filter_op = VolcanoFilter(scan, 0, 5)  # Filtrar onde col 0 > 5

filter_op.open()
results = []
while (row := filter_op.next()) is not None:
    results.append(row)
filter_op.close()

print("Resultado do filtro (coluna 0 > 5):")
print(results)

### Exercício 2: Compare o custo de chamadas de função

In [ ]:
# Medir o overhead puro de chamadas de função
def empty_function():
    pass

n_calls = 1_000_000

# Com chamadas de função
start = time.perf_counter()
for _ in range(n_calls):
    empty_function()
with_calls = time.perf_counter() - start

# Sem chamadas (loop vazio)
start = time.perf_counter()
for _ in range(n_calls):
    pass
without_calls = time.perf_counter() - start

overhead = (with_calls - without_calls) * 1e9 / n_calls  # nanosegundos por chamada

print(f"Overhead por chamada de função: ~{overhead:.1f} ns")
print(f"Em 1 milhão de linhas: ~{overhead * n_calls / 1e6:.1f} ms de overhead puro!")

## 1.9 Resumo do Capítulo

| Aspecto | Modelo Volcano | Modelo Vetorizado |
|---------|---------------|-------------------|
| Unidade de processamento | 1 linha | 1024-2048 linhas |
| Chamadas de função | N (uma por linha) | N/1024 |
| Cache efficiency | Baixa | Alta |
| SIMD | Não utiliza | Utiliza |
| Branch prediction | Ruim | Excelente |
| Complexidade | Simples | Moderada |

### Principais takeaways:
1. O modelo Volcano é elegante mas ineficiente para CPUs modernas
2. Vetorização reduz drasticamente o overhead de chamadas de função
3. Processar dados em chunks permite que eles fiquem no cache L1
4. O DuckDB escolhe tamanhos de chunk otimizados para o cache da CPU

In [ ]:
# Limpar recursos
con.close()